**Declarar dependencias y confirmar que funcionan**

In [1]:
import pandas as pd
import torch
print(torch.version)
print(torch.cuda.is_available())
from transformers.utils import is_torch_available, is_tf_available

print("Torch disponible:", is_torch_available())
print("TensorFlow disponible:", is_tf_available())

<module 'torch.version' from 'c:\\Users\\migue\\Documents\\GitHub\\Practica_BAIN_4\\.env\\Lib\\site-packages\\torch\\version.py'>
True


c:\Users\migue\Documents\GitHub\Practica_BAIN_4\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch disponible: True
TensorFlow disponible: False


**Referenciar el LLM y probar que funciona**

In [2]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)

messages = [
    [
        {
            "role": "system",
            "content": [
                {
                    "type": "text", 
                    "text": "Eres un asistente muy útil."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": "Escribe un poema sobre Hugging Face, la compañía"
                }
            ]
        }
    ]
]

output = pipe(messages, max_new_tokens=50)

output

Device set to use cuda


[[{'generated_text': [{'role': 'system',
     'content': [{'type': 'text', 'text': 'Eres un asistente muy útil.'}]},
    {'role': 'user',
     'content': [{'type': 'text',
       'text': 'Escribe un poema sobre Hugging Face, la compañía'}]},
    {'role': 'assistant',
     'content': 'Claro, aquí tienes un poema sobre Hugging Face:\n\nEn el mundo del aprendizaje profundo,\nDonde los modelos se expanden y florecen,\nNace Hugging Face, una fuerza genial,\nUn hogar para la inteligencia,'}]}]]

**Importar y limpiar el DataFrame**

In [3]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)

                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

**Crear las prompts**

In [52]:
prompts = { 'base' : None, 'zero_shot' : None, 'few_shots' : None, 'chain_of_thought' : None }

prompts['base'] = lambda opinion: f"Clasifica la siguiente opinión en base a como interpretes la actitud del mensaje, dicha actitud puede ser vista como positiva, negativa, neutral o desconocida: '{opinion}'. Responde con una sola palabra: 'positiva', 'negativa', 'neutral' o 'desconocida'."

prompts['zero_shot'] = lambda opinion: f"Clasifica la siguiente opinión como positiva, negativa, neutral o desconocida: '{opinion}'. Responde con una sola palabra: 'positiva', 'negativa', 'neutral' o 'desconocida'."

prompts['few_shots'] = lambda opinion: f"""
Aquí tienes algunas opiniones clasificadas:

Opinión: 'Me encanta este producto. Es increíble.' → Sentimiento: positiva
Opinión: 'Este servicio es terrible, nunca lo recomendaría.' → Sentimiento: negativa
Opinión: 'El artículo está bien, pero no es excepcional.' → Sentimiento: neutral

Clasifica esta opinión: '{opinion}'.

Responde con una sola palabra: 'positiva', 'negativa', 'neutral' o 'desconocida'.
"""

prompts['chain_of_thought'] = lambda opinion: f"""
Pensando en la siguiente opinión:

Opinión: '{opinion}'

La opinion puede ser positiva, negativa. neutral o desconocida. ¿Qué opinas?

Responde con una sola palabra: 'positiva', 'negativa', 'neutral' o 'desconocida'.
"""

**Clasificar cada opinion con cada prompt**

In [53]:
def classify(pipe, opinion, prompt):
    """
    Classify the sentiment of an opinion using a given prompt.
    """
    # Generate the prompt
    prompt_text = prompt(opinion)

    # Generate the message
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text", 
                    "text": "Eres un clasificador de sentimientos y respondes con una sola palabra: 'positivo', 'negativo', 'neutral' o 'desconocido'."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": prompt_text
                }
            ]
        }
    ]
    
    # Get the response from the model
    response = pipe(messages, max_new_tokens=50)

    # Extract the sentiment from the response
    sentiment = response[0]['generated_text'][2]['content']

    return sentiment

In [54]:
for row in df.iterrows():

    opinion = row[1]['opinion']
    label = row[1]['label']

    print(f"Opinion: {opinion} - Label: {label}")

    for prompt_name, prompt in prompts.items():

        # Classify the opinion using the prompt
        sentiment = classify(pipe, opinion, prompt).strip().lower()

        # Add the sentiment to the DataFrame at the column for the prompt used
        row_index = row[0]
        df.at[row_index, prompt_name] = sentiment

        # Print the result
        print(f"\t{prompt_name} sentiment: {sentiment}")

    print("")

Opinion: Es un gran servicio - Label: positiva
	base sentiment: positiva
	zero_shot sentiment: positiva
	few_shots sentiment: positiva
	chain_of_thought sentiment: positiva

Opinion: Es regular - Label: neutral
	base sentiment: neutral
	zero_shot sentiment: neutral
	few_shots sentiment: neutral
	chain_of_thought sentiment: neutral

Opinion: Es un producto útil - Label: positiva
	base sentiment: positiva
	zero_shot sentiment: positiva
	few_shots sentiment: positiva
	chain_of_thought sentiment: positiva

Opinion: Es un fraude - Label: negativa
	base sentiment: negativa
	zero_shot sentiment: negativa
	few_shots sentiment: negativa
	chain_of_thought sentiment: negativa

Opinion: Me gusta mucho - Label: positiva
	base sentiment: positiva
	zero_shot sentiment: positiva
	few_shots sentiment: positiva
	chain_of_thought sentiment: positiva

Opinion: Es un desastre - Label: negativa
	base sentiment: negativa
	zero_shot sentiment: negativa
	few_shots sentiment: negativa
	chain_of_thought sentimen

**Analizar la tasa de aciertos para cada prompt**

In [58]:
aciertos = {
    'base': (df['base'] == df['label']).sum(),
    'zero_shot': (df['zero_shot'] == df['label']).sum(),
    'few_shots': (df['few_shots'] == df['label']).sum(),
    'chain_of_thought': (df['chain_of_thought'] == df['label']).sum()
}

print(f"Aciertos con la prompt base: {aciertos['base']} - Ratio: {aciertos['base'] / len(df):.2f}")
print(f"Aciertos con la prompt zero shot: {aciertos['zero_shot']} - Ratio: {aciertos['zero_shot'] / len(df):.2f}")
print(f"Aciertos con la prompt few shots: {aciertos['few_shots']}- Ratio: {aciertos['few_shots'] / len(df):.2f}")
print(f"Aciertos con la prompt chain of thought: {aciertos['chain_of_thought']} - Ratio: {aciertos['chain_of_thought'] / len(df):.2f}")

Aciertos con la prompt base: 26 - Ratio: 0.87
Aciertos con la prompt zero shot: 25 - Ratio: 0.83
Aciertos con la prompt few shots: 30- Ratio: 1.00
Aciertos con la prompt chain of thought: 30 - Ratio: 1.00
